### **Explicación del problema**
El principio de **Indirection** establece que **se deben utilizar intermediarios adecuados para reducir el acoplamiento directo entre componentes que no deberían depender entre sí**.  
En este caso, crearemos un **sistema de notificaciones** donde diferentes servicios (Email, SMS y Push) **están directamente acoplados a la clase `User`**, en lugar de usar un intermediario para gestionar las notificaciones.  

#### **Problemas del diseño (Violación de Indirection)**
- La clase `User` **está directamente acoplada** a `EmailNotification`, `SMSNotification` y `PushNotification`, lo que **dificulta la escalabilidad y mantenimiento**.
- Si queremos **agregar un nuevo tipo de notificación**, debemos modificar `User`, lo que rompe el principio **Open/Closed (OCP)**.
- La lógica de envío de notificaciones está mezclada dentro de `User`, en lugar de usar una **capa intermedia**.

---

### **Código con Violación del Principio de Indirection**
```python
from typing import List

class EmailNotification:
    """Notificación por correo electrónico."""
    def send(self, email: str, message: str) -> None:
        print(f"Enviando email a {email}: {message}")

class SMSNotification:
    """Notificación por SMS."""
    def send(self, phone: str, message: str) -> None:
        print(f"Enviando SMS a {phone}: {message}")

class PushNotification:
    """Notificación por Push."""
    def send(self, device_id: str, message: str) -> None:
        print(f"Enviando notificación Push a {device_id}: {message}")

class User:
    """Clase de usuario con envío de notificaciones acoplado a clases concretas."""
    def __init__(self, name: str, email: str, phone: str, device_id: str) -> None:
        self.name = name
        self.email = email
        self.phone = phone
        self.device_id = device_id

    def notify(self, message: str) -> None:
        """Envía notificaciones usando cada canal disponible."""
        email_service = EmailNotification()
        sms_service = SMSNotification()
        push_service = PushNotification()
        
        email_service.send(self.email, message)
        sms_service.send(self.phone, message)
        push_service.send(self.device_id, message)

# Uso del código
user = User("Juan Pérez", "juan@example.com", "555-1234", "device_001")
user.notify("¡Bienvenido a nuestra plataforma!")
```

---

### **¿Por qué este código viola el principio de Indirection?**
1. **El usuario está acoplado directamente a múltiples servicios de notificación**:
   - `User` tiene conocimiento explícito de `EmailNotification`, `SMSNotification` y `PushNotification`.
   - Si queremos cambiar la forma de envío, tenemos que modificar `User`.

2. **No hay una capa de intermediación que gestione la notificación**:
   - **Falta un servicio intermedio** que abstraiga la lógica de notificación y desacople `User` de los métodos de envío.
   - `User` debería delegar la notificación a una capa de servicio, en lugar de hacerlo directamente.

3. **Dificultad para extender el sistema**:
   - Si queremos agregar `WhatsAppNotification`, hay que modificar `User`, lo que no es escalable.

---

## **Versión Refactorizada con el Principio de Indirection**
Ahora aplicamos **Indirection** creando una capa intermedia: `NotificationService`, que gestiona el envío de notificaciones.

```python
from typing import Protocol, List

class NotificationChannel(Protocol):
    """Interfaz común para los métodos de notificación."""
    def send(self, recipient: str, message: str) -> None:
        ...

class EmailNotification(NotificationChannel):
    """Notificación por correo electrónico."""
    def send(self, recipient: str, message: str) -> None:
        print(f"Enviando email a {recipient}: {message}")

class SMSNotification(NotificationChannel):
    """Notificación por SMS."""
    def send(self, recipient: str, message: str) -> None:
        print(f"Enviando SMS a {recipient}: {message}")

class PushNotification(NotificationChannel):
    """Notificación por Push."""
    def send(self, recipient: str, message: str) -> None:
        print(f"Enviando notificación Push a {recipient}: {message}")

class NotificationService:
    """Capa intermedia para manejar notificaciones sin acoplarlas a User."""
    def __init__(self, channels: List[NotificationChannel]) -> None:
        self.channels = channels

    def notify(self, recipient: str, message: str) -> None:
        """Envía notificaciones a través de todos los canales disponibles."""
        for channel in self.channels:
            channel.send(recipient, message)

class User:
    """Clase de usuario desacoplada del sistema de notificaciones."""
    def __init__(self, name: str, email: str, phone: str, device_id: str, notification_service: NotificationService) -> None:
        self.name = name
        self.email = email
        self.phone = phone
        self.device_id = device_id
        self.notification_service = notification_service

    def notify(self, message: str) -> None:
        """Notifica al usuario a través del servicio de notificación."""
        self.notification_service.notify(self.email, message)

# Configuración de los canales de notificación
channels = [EmailNotification(), SMSNotification(), PushNotification()]
notification_service = NotificationService(channels)

# Uso del código refactorizado
user = User("Juan Pérez", "juan@example.com", "555-1234", "device_001", notification_service)
user.notify("¡Bienvenido a nuestra plataforma!")
```

---

### **¿Por qué esta versión respeta el principio de Indirection?**
✅ **Se introduce una capa de abstracción (`NotificationService`)**  
   - `User` ya no se comunica directamente con `EmailNotification`, `SMSNotification` o `PushNotification`.  
   - Ahora delega la responsabilidad a `NotificationService`, lo que **reduce el acoplamiento**.

✅ **Extensible sin modificar `User`**  
   - Si queremos agregar un nuevo canal de notificación (`WhatsAppNotification`), solo implementamos `NotificationChannel` y lo añadimos a `NotificationService`.

✅ **Fácil de modificar y probar**  
   - `User` y `NotificationService` pueden probarse de forma independiente.  
   - Podemos inyectar **notificaciones simuladas (mock objects)** para pruebas unitarias.

✅ **Cumple el principio Open/Closed (OCP)**  
   - `User` **no necesita modificarse** para admitir nuevos métodos de notificación.

---

### **Comparación antes y después**
| Problema              | Código original 🚨 | Código refactorizado ✅ |
|-----------------------|------------------|------------------|
| **Acoplamiento** | `User` conoce todas las clases concretas de notificación. | `User` solo conoce `NotificationService`. |
| **Extensibilidad** | Agregar un nuevo método de notificación implica modificar `User`. | Se puede agregar un nuevo canal sin tocar `User`. |
| **Modularidad** | `User` mezcla lógica de usuario con lógica de notificación. | `User` solo delega la notificación a `NotificationService`. |
| **Mantenimiento** | Difícil de mantener porque `User` tiene demasiadas dependencias. | Fácil de mantener porque `NotificationService` centraliza la lógica. |

---

### **Conclusión**
El código original **violaba Indirection** porque `User` dependía directamente de las clases concretas de notificación, sin un intermediario.  
La versión refactorizada introduce una **capa intermedia (`NotificationService`)**, que desacopla `User` de los métodos de notificación, haciendo el código **más escalable, flexible y fácil de mantener**. 🚀

In [2]:
from typing import List

class EmailNotification:
    """Notificación por correo electrónico."""
    def send(self, email: str, message: str) -> None:
        print(f"Enviando email a {email}: {message}")

class SMSNotification:
    """Notificación por SMS."""
    def send(self, phone: str, message: str) -> None:
        print(f"Enviando SMS a {phone}: {message}")

class PushNotification:
    """Notificación por Push."""
    def send(self, device_id: str, message: str) -> None:
        print(f"Enviando notificación Push a {device_id}: {message}")

class User:
    """Clase de usuario con envío de notificaciones acoplado a clases concretas."""
    def __init__(self, name: str, email: str, phone: str, device_id: str) -> None:
        self.name = name
        self.email = email
        self.phone = phone
        self.device_id = device_id

    def notify(self, message: str) -> None:
        """Envía notificaciones usando cada canal disponible."""
        email_service = EmailNotification()
        sms_service = SMSNotification()
        push_service = PushNotification()
        
        email_service.send(self.email, message)
        sms_service.send(self.phone, message)
        push_service.send(self.device_id, message)

# Uso del código
user = User("Juan Pérez", "juan@example.com", "555-1234", "device_001")
user.notify("¡Bienvenido a nuestra plataforma!")


Enviando email a juan@example.com: ¡Bienvenido a nuestra plataforma!
Enviando SMS a 555-1234: ¡Bienvenido a nuestra plataforma!
Enviando notificación Push a device_001: ¡Bienvenido a nuestra plataforma!


In [1]:
from typing import Protocol, List

class NotificationChannel(Protocol):
    """Interfaz común para los métodos de notificación."""
    def send(self, recipient: str, message: str) -> None:
        ...

class EmailNotification(NotificationChannel):
    """Notificación por correo electrónico."""
    def send(self, recipient: str, message: str) -> None:
        print(f"Enviando email a {recipient}: {message}")

class SMSNotification(NotificationChannel):
    """Notificación por SMS."""
    def send(self, recipient: str, message: str) -> None:
        print(f"Enviando SMS a {recipient}: {message}")

class PushNotification(NotificationChannel):
    """Notificación por Push."""
    def send(self, recipient: str, message: str) -> None:
        print(f"Enviando notificación Push a {recipient}: {message}")

class NotificationService:
    """Capa intermedia para manejar notificaciones sin acoplarlas a User."""
    def __init__(self, channels: List[NotificationChannel]) -> None:
        self.channels = channels

    def notify(self, recipient: str, message: str) -> None:
        """Envía notificaciones a través de todos los canales disponibles."""
        for channel in self.channels:
            channel.send(recipient, message)

class User:
    """Clase de usuario desacoplada del sistema de notificaciones."""
    def __init__(self, name: str, email: str, phone: str, device_id: str, notification_service: NotificationService) -> None:
        self.name = name
        self.email = email
        self.phone = phone
        self.device_id = device_id
        self.notification_service = notification_service

    def notify(self, message: str) -> None:
        """Notifica al usuario a través del servicio de notificación."""
        self.notification_service.notify(self.email, message)

# Configuración de los canales de notificación
channels = [EmailNotification(), SMSNotification(), PushNotification()]
notification_service = NotificationService(channels)

# Uso del código refactorizado
user = User("Juan Pérez", "juan@example.com", "555-1234", "device_001", notification_service)
user.notify("¡Bienvenido a nuestra plataforma!")


Enviando email a juan@example.com: ¡Bienvenido a nuestra plataforma!
Enviando SMS a juan@example.com: ¡Bienvenido a nuestra plataforma!
Enviando notificación Push a juan@example.com: ¡Bienvenido a nuestra plataforma!


Hay que tener en cuenta que **es extraño y problemático** tener el método `notify` dentro de `User`, porque **la notificación no es una responsabilidad directa del usuario**.  

En el diseño refactorizado, aunque hemos mejorado la separación de responsabilidades introduciendo `NotificationService`, **User aún tiene conocimiento de la existencia de las notificaciones**, lo cual **sigue siendo una ligera violación del principio de Indirection**.

---

## **Corrección: Eliminar `notify` de `User`**
Para mejorar aún más el diseño, `User` **debería ser un simple modelo de datos** y no saber nada sobre cómo se notifican los eventos. **El servicio de notificación debería encargarse de todo**.

Aquí está el código corregido:

---

### **Código Mejorado**
```python
from typing import Protocol, List

class NotificationChannel(Protocol):
    """Interfaz común para los métodos de notificación."""
    def send(self, recipient: str, message: str) -> None:
        ...

class EmailNotification(NotificationChannel):
    """Notificación por correo electrónico."""
    def send(self, recipient: str, message: str) -> None:
        print(f"Enviando email a {recipient}: {message}")

class SMSNotification(NotificationChannel):
    """Notificación por SMS."""
    def send(self, recipient: str, message: str) -> None:
        print(f"Enviando SMS a {recipient}: {message}")

class PushNotification(NotificationChannel):
    """Notificación por Push."""
    def send(self, recipient: str, message: str) -> None:
        print(f"Enviando notificación Push a {recipient}: {message}")

class NotificationService:
    """Capa intermedia para manejar notificaciones sin acoplarlas a User."""
    def __init__(self, channels: List[NotificationChannel]) -> None:
        self.channels = channels

    def notify_user(self, user: "User", message: str) -> None:
        """Envía notificaciones a través de todos los canales disponibles."""
        for channel in self.channels:
            channel.send(user.email, message)  # Se puede modificar para elegir el canal apropiado

class User:
    """Modelo de datos de usuario sin lógica de notificación."""
    def __init__(self, name: str, email: str, phone: str, device_id: str) -> None:
        self.name = name
        self.email = email
        self.phone = phone
        self.device_id = device_id

# Configuración de los canales de notificación
channels = [EmailNotification(), SMSNotification(), PushNotification()]
notification_service = NotificationService(channels)

# Uso del código refactorizado
user = User("Juan Pérez", "juan@example.com", "555-1234", "device_001")
notification_service.notify_user(user, "¡Bienvenido a nuestra plataforma!")
```

---

### **¿Por qué este diseño es mejor?**
✅ **Elimina `notify` de `User`**, porque **notificar no es responsabilidad de un usuario**.  
✅ **El usuario ahora es un modelo de datos puro**, sin dependencias de notificación.  
✅ **El `NotificationService` maneja toda la lógica**, eliminando cualquier acoplamiento en `User`.  
✅ **El diseño es más flexible**, porque ahora podríamos agregar reglas para elegir qué canal usar dependiendo del usuario.

---

### **Conclusión**
**ANTES:** `User` tenía un método `notify()`, lo que implicaba que **sabía que las notificaciones existían y cómo funcionaban**.  
**DESPUÉS:** `User` es un **modelo puro de datos**, mientras que `NotificationService` es la capa que decide **cuándo y cómo notificar a los usuarios**.

Esta versión es **totalmente compatible con el principio de Indirection**, asegurando que los cambios en notificaciones **nunca afecten a `User`**. 🚀

In [3]:
from typing import Protocol, List

class NotificationChannel(Protocol):
    """Interfaz común para los métodos de notificación."""
    def send(self, recipient: str, message: str) -> None:
        ...

class EmailNotification(NotificationChannel):
    """Notificación por correo electrónico."""
    def send(self, recipient: str, message: str) -> None:
        print(f"Enviando email a {recipient}: {message}")

class SMSNotification(NotificationChannel):
    """Notificación por SMS."""
    def send(self, recipient: str, message: str) -> None:
        print(f"Enviando SMS a {recipient}: {message}")

class PushNotification(NotificationChannel):
    """Notificación por Push."""
    def send(self, recipient: str, message: str) -> None:
        print(f"Enviando notificación Push a {recipient}: {message}")

class NotificationService:
    """Capa intermedia para manejar notificaciones sin acoplarlas a User."""
    def __init__(self, channels: List[NotificationChannel]) -> None:
        self.channels = channels

    def notify_user(self, user: "User", message: str) -> None:
        """Envía notificaciones a través de todos los canales disponibles."""
        for channel in self.channels:
            channel.send(user.email, message)  # Se puede modificar para elegir el canal apropiado

class User:
    """Modelo de datos de usuario sin lógica de notificación."""
    def __init__(self, name: str, email: str, phone: str, device_id: str) -> None:
        self.name = name
        self.email = email
        self.phone = phone
        self.device_id = device_id

# Configuración de los canales de notificación
channels = [EmailNotification(), SMSNotification(), PushNotification()]
notification_service = NotificationService(channels)

# Uso del código refactorizado
user = User("Juan Pérez", "juan@example.com", "555-1234", "device_001")
notification_service.notify_user(user, "¡Bienvenido a nuestra plataforma!")


Enviando email a juan@example.com: ¡Bienvenido a nuestra plataforma!
Enviando SMS a juan@example.com: ¡Bienvenido a nuestra plataforma!
Enviando notificación Push a juan@example.com: ¡Bienvenido a nuestra plataforma!


# Más lugar a mejora?

Sí, aunque el diseño es **mucho mejor** que la versión inicial, todavía hay **algunas oportunidades de mejora** desde la perspectiva de diseño y principios de arquitectura de software. Vamos a analizarlas y corregirlas.

---

## **Problemas que aún persisten**
1. **El servicio de notificaciones está acoplado a `User`**  
   - `NotificationService` sigue dependiendo de los atributos de `User` (`email`, `phone`, `device_id`).
   - Si `User` cambia su estructura o queremos notificar a otros tipos de entidades (por ejemplo, `Admin` o `Customer`), tendremos que modificar `NotificationService`.

2. **La lógica de selección del canal está en `NotificationService`**  
   - `NotificationService` **envía todas las notificaciones** sin decidir cuál canal es el más adecuado.
   - Podría haber usuarios que prefieren recibir solo SMS, solo email, o una combinación.

3. **Falta una abstracción para representar destinatarios de notificaciones**  
   - Actualmente, `NotificationService` usa `User`, pero podríamos necesitar enviar notificaciones a otras entidades (por ejemplo, `Admin`, `Supplier`, etc.).
   - Esto nos obliga a depender de una clase concreta (`User`), lo que **viola el principio de Dependency Inversion (DIP)**.

---

## **Solución: Desacoplar `User` y mejorar la gestión de canales**
En esta versión mejorada:
✅ Introducimos una **interfaz `Notifiable`** para representar cualquier entidad que pueda recibir notificaciones.  
✅ `User` implementa esta interfaz en lugar de ser referenciado directamente por `NotificationService`.  
✅ Se agregan **preferencias de notificación**, permitiendo a cada usuario definir qué canal desea usar.  
✅ `NotificationService` ahora decide **qué canal usar para cada usuario**, en lugar de enviar todas las notificaciones.

---

### **Código Mejorado**
```python
from typing import Protocol, List, Dict

class NotificationChannel(Protocol):
    """Interfaz común para los métodos de notificación."""
    def send(self, recipient: str, message: str) -> None:
        ...

class EmailNotification(NotificationChannel):
    """Notificación por correo electrónico."""
    def send(self, recipient: str, message: str) -> None:
        print(f"📧 Enviando email a {recipient}: {message}")

class SMSNotification(NotificationChannel):
    """Notificación por SMS."""
    def send(self, recipient: str, message: str) -> None:
        print(f"📱 Enviando SMS a {recipient}: {message}")

class PushNotification(NotificationChannel):
    """Notificación por Push."""
    def send(self, recipient: str, message: str) -> None:
        print(f"🔔 Enviando notificación Push a {recipient}: {message}")

class Notifiable(Protocol):
    """Interfaz que define a un destinatario de notificaciones."""
    def get_notification_preferences(self) -> Dict[str, str]:
        """Devuelve las preferencias de notificación del usuario."""
        ...

class User:
    """Modelo de usuario que implementa Notifiable."""
    def __init__(self, name: str, email: str, phone: str, device_id: str, preferences: List[str]) -> None:
        self.name = name
        self.email = email
        self.phone = phone
        self.device_id = device_id
        self.preferences = preferences  # Lista de canales que el usuario quiere recibir

    def get_notification_preferences(self) -> Dict[str, str]:
        """Retorna los datos del usuario según sus preferencias de notificación."""
        available_methods = {
            "email": self.email,
            "sms": self.phone,
            "push": self.device_id
        }
        return {method: available_methods[method] for method in self.preferences if method in available_methods}

class NotificationService:
    """Maneja el envío de notificaciones usando un canal apropiado."""
    def __init__(self, channels: Dict[str, NotificationChannel]) -> None:
        self.channels = channels  # Diccionario {nombre_canal: instancia_canal}

    def notify(self, recipient: Notifiable, message: str) -> None:
        """Envía la notificación solo por los canales que el usuario ha configurado."""
        preferences = recipient.get_notification_preferences()

        if not preferences:
            print("⚠️ El usuario no tiene canales de notificación configurados.")
            return

        for method, contact in preferences.items():
            if method in self.channels:
                self.channels[method].send(contact, message)

# Configuración de los canales de notificación
channels = {
    "email": EmailNotification(),
    "sms": SMSNotification(),
    "push": PushNotification()
}
notification_service = NotificationService(channels)

# Creación de usuarios con diferentes preferencias
user1 = User("Juan Pérez", "juan@example.com", "555-1234", "device_001", ["email", "push"])
user2 = User("María López", "maria@example.com", "555-5678", "device_002", ["sms"])

# Envío de notificaciones según las preferencias del usuario
notification_service.notify(user1, "🎉 ¡Bienvenido a nuestra plataforma!")
notification_service.notify(user2, "📢 Tienes una nueva promoción disponible.")
```

---

## **¿Por qué este diseño es mejor?**
✅ **Elimina la dependencia directa entre `User` y `NotificationService`**  
   - `NotificationService` ya no depende de `User`, sino de cualquier objeto que implemente `Notifiable`.  
   - Esto hace que el sistema sea **más flexible y abierto a nuevos tipos de destinatarios**.

✅ **Introduce el concepto de "preferencias de notificación"**  
   - Ahora, los usuarios pueden decidir **qué tipo de notificación quieren recibir**.  
   - Si `User2` solo quiere SMS, no recibirá email ni push.

✅ **Facilita la extensión del sistema**  
   - Si queremos agregar `WhatsAppNotification`, solo hay que implementar `NotificationChannel` y actualizar el diccionario de canales.

✅ **Cumple completamente con el principio de Indirection**  
   - `User` ya no sabe nada sobre cómo se envían las notificaciones.  
   - `NotificationService` actúa como una capa intermedia inteligente que **elige automáticamente los canales adecuados**.

---

## **Comparación Antes vs. Después**
| Problema              | Código inicial 🚨 | Código mejorado ✅ |
|-----------------------|------------------|------------------|
| **Acoplamiento** | `User` dependía de clases concretas de notificación. | `User` solo implementa `Notifiable`, sin conocer los métodos de notificación. |
| **Escalabilidad** | Agregar nuevos canales requería modificar `NotificationService`. | Se pueden agregar canales sin tocar `User` o `NotificationService`. |
| **Personalización** | Todos los usuarios recibían todas las notificaciones. | Cada usuario puede elegir sus canales de preferencia. |
| **Flexibilidad** | `User` estaba limitado a recibir notificaciones. | Ahora podemos notificar a `Admin`, `Supplier` o cualquier entidad que implemente `Notifiable`. |

---

## **Conclusión**
### 🚨 **Problema Inicial**
El sistema violaba **Indirection** porque `User` estaba acoplado a las clases de notificación y `NotificationService` tenía lógica rígida.

### ✅ **Solución Mejorada**
- Se introdujo **una interfaz `Notifiable`**, haciendo que `User` solo defina cómo se le debe notificar.  
- Se añadió **un diccionario de canales de notificación**, permitiendo cambiar la lógica sin modificar `NotificationService`.  
- Se agregaron **preferencias de notificación**, haciendo el sistema más personalizado y flexible.

Este diseño **cumple completamente con el principio de Indirection**, haciendo que el sistema sea **modular, flexible y fácil de extender**. 🚀

In [4]:
from typing import Protocol, List, Dict

class NotificationChannel(Protocol):
    """Interfaz común para los métodos de notificación."""
    def send(self, recipient: str, message: str) -> None:
        ...

class EmailNotification(NotificationChannel):
    """Notificación por correo electrónico."""
    def send(self, recipient: str, message: str) -> None:
        print(f"📧 Enviando email a {recipient}: {message}")

class SMSNotification(NotificationChannel):
    """Notificación por SMS."""
    def send(self, recipient: str, message: str) -> None:
        print(f"📱 Enviando SMS a {recipient}: {message}")

class PushNotification(NotificationChannel):
    """Notificación por Push."""
    def send(self, recipient: str, message: str) -> None:
        print(f"🔔 Enviando notificación Push a {recipient}: {message}")

class Notifiable(Protocol):
    """Interfaz que define a un destinatario de notificaciones."""
    def get_notification_preferences(self) -> Dict[str, str]:
        """Devuelve las preferencias de notificación del usuario."""
        ...

class User:
    """Modelo de usuario que implementa Notifiable."""
    def __init__(self, name: str, email: str, phone: str, device_id: str, preferences: List[str]) -> None:
        self.name = name
        self.email = email
        self.phone = phone
        self.device_id = device_id
        self.preferences = preferences  # Lista de canales que el usuario quiere recibir

    def get_notification_preferences(self) -> Dict[str, str]:
        """Retorna los datos del usuario según sus preferencias de notificación."""
        available_methods = {
            "email": self.email,
            "sms": self.phone,
            "push": self.device_id
        }
        return {method: available_methods[method] for method in self.preferences if method in available_methods}

class NotificationService:
    """Maneja el envío de notificaciones usando un canal apropiado."""
    def __init__(self, channels: Dict[str, NotificationChannel]) -> None:
        self.channels = channels  # Diccionario {nombre_canal: instancia_canal}

    def notify(self, recipient: Notifiable, message: str) -> None:
        """Envía la notificación solo por los canales que el usuario ha configurado."""
        preferences = recipient.get_notification_preferences()

        if not preferences:
            print("⚠️ El usuario no tiene canales de notificación configurados.")
            return

        for method, contact in preferences.items():
            if method in self.channels:
                self.channels[method].send(contact, message)

# Configuración de los canales de notificación
channels = {
    "email": EmailNotification(),
    "sms": SMSNotification(),
    "push": PushNotification()
}
notification_service = NotificationService(channels)

# Creación de usuarios con diferentes preferencias
user1 = User("Juan Pérez", "juan@example.com", "555-1234", "device_001", ["email", "push"])
user2 = User("María López", "maria@example.com", "555-5678", "device_002", ["sms"])

# Envío de notificaciones según las preferencias del usuario
notification_service.notify(user1, "🎉 ¡Bienvenido a nuestra plataforma!")
notification_service.notify(user2, "📢 Tienes una nueva promoción disponible.")


📧 Enviando email a juan@example.com: 🎉 ¡Bienvenido a nuestra plataforma!
🔔 Enviando notificación Push a device_001: 🎉 ¡Bienvenido a nuestra plataforma!
📱 Enviando SMS a 555-5678: 📢 Tienes una nueva promoción disponible.
